# Deepface Face Verification test 

- Import relevant libraries:

In [149]:
# !pip install deepface
from deepface import DeepFace
from deepface_test_functions import find_image, deepface_benchmark_lfw_split, deepface_benchmark, deepface_get_accuracy, deepface_run_dataset

import matplotlib.pyplot as plt
import pandas as pd                 
import regex as re

import os

# Benchmark Datasets
benchmark_df = pd.read_csv('./data/LFW-csv/pairs.csv')
benchmark_balance = pd.read_csv('./data/LFW-csv/pairs_balance.csv')
# benchmark_balance_mod = pd.read_csv('./data/LFW-csv/pairs_balance_mod.csv')
benchmark_deepface_df = pd.read_csv('./data/LFW-csv/pairs_benchmark_deepface.csv')


# Results:
result_df = pd.read_csv('./result/result_deepface.csv')
result_benchmark_gender = pd.read_csv('./result/deepface_benchmark_gender.csv')
# result_benchmark_mod = pd.read_csv('./result/deepface_benchmark_mod.csv')

result_benchmark_gender_nonperturbed = pd.read_csv('./result/deepface_benchmark_gender_nonperturbed.csv')
result_benchmark_gender_perturbed = pd.read_csv('./result/deepface_benchmark_gender_perturbed.csv')

In [2]:
result_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,TP,TN,FP,FN,Undetected
0,Facenet,LFW,79.0,100.0,58.0,6000,1733,2992,8,1267,0.0
1,VGG-Face,LFW,86.0,81.0,93.0,6000,2799,2334,666,201,0.0
2,OpenFace,LFW,56.0,86.0,15.0,6000,441,2929,71,2559,0.0
3,DeepFace,LFW,64.0,76.0,41.0,6000,1225,2621,379,1775,0.0
4,ArcFace,LFW,94.0,99.0,89.0,6000,2678,2966,34,322,0.0


In [3]:
result_benchmark_gender

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender,92.0,99.0,84.0,1500,Male,638.0,735.0,8.0,119.0,0.0
1,Facenet512,LFW_gender,89.0,99.0,80.0,1500,Female,613.0,728.0,6.0,153.0,0.0
2,Facenet,LFW_gender,72.0,100.0,45.0,1500,Male,342.0,742.0,1.0,415.0,0.0
3,Facenet,LFW_gender,71.0,99.0,44.0,1500,Female,340.0,732.0,2.0,426.0,0.0
4,VGG-Face,LFW_gender,86.0,87.0,84.0,1500,Male,639.0,647.0,96.0,118.0,0.0
5,VGG-Face,LFW_gender,84.0,98.0,71.0,1500,Female,544.0,723.0,11.0,222.0,0.0
6,OpenFace,LFW_gender,52.0,90.0,5.0,1500,Male,38.0,739.0,4.0,719.0,0.0
7,OpenFace,LFW_gender,56.0,96.0,15.0,1500,Female,115.0,729.0,5.0,651.0,0.0
8,DeepFace,LFW_gender,58.0,90.0,19.0,1500,Male,147.0,727.0,16.0,610.0,0.0
9,DeepFace,LFW_gender,58.0,96.0,18.0,1500,Female,139.0,728.0,6.0,627.0,0.0


# Deepface Threshold results:

In [4]:
## Results based on https://youtu.be/i_MOwvhbLdI 
_vgg_face = {"Model": "VGGFace", "Accuracy": 89.28, "Precision": 97.41, "Recall": 80.71}
_facenet = {"Model": "FaceNet", "Accuracy": 98.21, "Precision": 100, "Recall": 96.42}
_openface = {"Model": "OpenFace", "Accuracy": 57.85, "Precision": 95.83, "Recall": 16.42}
_deepface = {"Model": "DeepFace", "Accuracy": 54.64, "Precision": 100, "Recall": 9.28}

models_results = [_facenet, _openface, _deepface]

result_model_df = pd.DataFrame([_vgg_face])
for model in models_results:
    result_model = pd.DataFrame([model])
    frames = [result_model_df, result_model]
    result_model_df = pd.concat(frames)

    result_model_df = result_model_df.reset_index()
    result_model_df.drop(columns=['index'], inplace=True)

result_model_df

,Model,Accuracy,Precision,Recall
0,VGGFace,89.28,97.41,80.71
1,FaceNet,98.21,100.00,96.42
2,OpenFace,57.85,95.83,16.42
3,DeepFace,54.64,100.00,9.28


## Paramater settings:

In [144]:
# select a DATASET from data folder:
# LFW, LFW-original
DATASET = "LFW"

# test with gender split LFW
# LFW_gender, LFW_gender_makeup_heavy
DATASET_LFW_SPLIT = "LFW_gender"

# select a model to run:
# models = ["Facenet512", "Facenet", "VGG-Face", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
MODEL = "DeepFace"

# Faces are represented as vectors, a face pair of same person should be similar, calculated by different metrics
# metrics = ["cosine", "euclidean", "euclidean_l2"]
METRIC = "euclidean_l2"

# Face detector, backend that helps aligning the face for better accuracy
# backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']
BACKEND = 'opencv'

# Select 'Male' or 'Female'
TEST_GENDER = 'Female'

# Test additional data directory:
# dir_path = f"./additional-data/{DATASET_TEST}"

# Test dataset directory:
dir_path = f"./data/{DATASET_LFW_SPLIT}"

## Deepface test: 

- Manual test:

In [6]:
# benchmark testing
# deepface_benchmark_res = deepface_benchmark(DATASET, dir_path, benchmark_df, MODEL, METRIC, BACKEND)

- Benchmark testing with LFW gender split

In [145]:
# benchmark testing LFW with gender split 
# benchmark_balance
# benchmark_balance_mod

# currrently using:  
# benchmark_deepface_df

deepface_benchmark_res = deepface_benchmark_lfw_split(DATASET_LFW_SPLIT, dir_path, benchmark_deepface_df,\
                                                                     MODEL, METRIC, BACKEND, TEST_GENDER)


1/1 [==============================] - 0s 43ms/step


## Result of the Deepface test:

- Manual testing:

- Benchmark testing:

In [146]:
# confusion matrix accuracy:
cm_acc = deepface_benchmark_res["CM_ACC"]
cm_pre = deepface_benchmark_res["Precision"]
cm_rec = deepface_benchmark_res["Recall"]
truth_pos = deepface_benchmark_res["TP"]
truth_neg = deepface_benchmark_res["TN"]
false_pos = deepface_benchmark_res["FP"]
false_neg = deepface_benchmark_res["FN"]
undetected = deepface_benchmark_res["Undetected"]

# total images
total_imgs = deepface_benchmark_res["Total Images"]

print(f"\
Accuracy of {MODEL} is {cm_acc}% out of {total_imgs} images from {DATASET} dataset.\n\
The Precision is {cm_pre} and Recall is {cm_rec}\n\
===================\n\
Truth Positive: {truth_pos} \n\
Truth Negative: {truth_neg} \n\
False Positive: {false_pos} \n\
False Negative: {false_neg} \n\
===================\n\
Undetected: {undetected}")

Accuracy of DeepFace is 57.99999999999999% out of 1000 images from LFW dataset.
The Precision is 84.0 and Recall is 20.0
Truth Positive: 102 
Truth Negative: 480 
False Positive: 20 
False Negative: 398 
Undetected: 0


# Saving result:

- Write to CSV for Benchmark LFW gender split

In [147]:
result_df_new = pd.DataFrame([deepface_benchmark_res])

# result_benchmark_gender
# result_benchmark_gender_nonperturbed
frames = [result_benchmark_gender_nonperturbed, result_df_new]
result_benchmark_df = pd.concat(frames)

result_benchmark_df = result_benchmark_df.reset_index()
result_benchmark_df.drop(columns=['index'], inplace=True)
result_benchmark_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender,92.0,97.0,85.0,1000,Male,426,489,11,74,0
1,Facenet512,LFW_gender,91.0,100.0,83.0,1000,Female,414,500,0,86,0
2,Facenet,LFW_gender,73.0,100.0,46.0,1000,Male,232,500,0,268,0
3,Facenet,LFW_gender,73.0,100.0,46.0,1000,Female,232,500,0,268,0
4,VGG-Face,LFW_gender,73.0,68.0,86.0,1000,Male,429,300,200,71,0
5,VGG-Face,LFW_gender,86.0,96.0,75.0,1000,Female,374,485,15,126,0
6,OpenFace,LFW_gender,53.0,94.0,6.0,1000,Male,31,498,2,469,0
7,OpenFace,LFW_gender,58.0,100.0,17.0,1000,Female,83,500,0,417,0
8,DeepFace,LFW_gender,56.0,65.0,23.0,1000,Male,117,438,62,383,0
9,ArcFace,LFW_gender,90.0,100.0,81.0,1000,Male,403,498,2,97,0


In [148]:
# Write results to csv file
# result_benchmark_df.to_csv("./result/deepface_benchmark_gender.csv", index=False)
# result_benchmark_gender = pd.read_csv('./result/deepface_benchmark_gender.csv')

result_benchmark_df.to_csv("./result/deepface_benchmark_gender_nonperturbed.csv", index=False)
result_benchmark_gender_nonperturbed = pd.read_csv('./result/deepface_benchmark_gender_nonperturbed.csv')

In [151]:
result_benchmark_gender_nonperturbed

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender,92.0,97.0,85.0,1000,Male,426,489,11,74,0
1,Facenet512,LFW_gender,91.0,100.0,83.0,1000,Female,414,500,0,86,0
2,Facenet,LFW_gender,73.0,100.0,46.0,1000,Male,232,500,0,268,0
3,Facenet,LFW_gender,73.0,100.0,46.0,1000,Female,232,500,0,268,0
4,VGG-Face,LFW_gender,73.0,68.0,86.0,1000,Male,429,300,200,71,0
5,VGG-Face,LFW_gender,86.0,96.0,75.0,1000,Female,374,485,15,126,0
6,OpenFace,LFW_gender,53.0,94.0,6.0,1000,Male,31,498,2,469,0
7,OpenFace,LFW_gender,58.0,100.0,17.0,1000,Female,83,500,0,417,0
8,DeepFace,LFW_gender,56.0,65.0,23.0,1000,Male,117,438,62,383,0
9,DeepFace,LFW_gender,58.0,84.0,20.0,1000,Female,102,480,20,398,0


- Write to CSV for perturbed data Benchmark LFW gender split

In [64]:
result_df_new = pd.DataFrame([deepface_benchmark_res])

frames = [result_benchmark_gender_perturbed, result_df_new]
result_benchmark_df = pd.concat(frames)

result_benchmark_df = result_benchmark_df.reset_index()
result_benchmark_df.drop(columns=['index'], inplace=True)
result_benchmark_df
# result_df_new

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender_makeup_heavy,87.0,94.0,80.0,1000,Male,398,476,24,102,0
1,Facenet512,LFW_gender_makeup_heavy,86.0,99.0,73.0,1000,Female,366,498,2,134,0
2,Facenet,LFW_gender_makeup_heavy,67.0,100.0,35.0,1000,Male,174,500,0,326,0
3,Facenet,LFW_gender_makeup_heavy,69.0,100.0,38.0,1000,Female,192,500,0,308,0
4,VGG-Face,LFW_gender_makeup_heavy,73.0,70.0,81.0,1000,Male,404,327,173,96,0
5,VGG-Face,LFW_gender_makeup_heavy,83.0,96.0,69.0,1000,Female,346,487,13,154,0
6,OpenFace,LFW_gender_makeup_heavy,54.0,95.0,8.0,1000,Male,38,498,2,462,0
7,OpenFace,LFW_gender_makeup_heavy,56.0,100.0,12.0,1000,Female,60,500,0,440,0
8,DeepFace,LFW_gender_makeup_heavy,55.0,62.0,25.0,1000,Male,127,422,78,373,0
9,DeepFace,LFW_gender_makeup_heavy,58.0,79.0,22.0,1000,Female,110,471,29,390,0


In [65]:
# Write results to csv file
result_benchmark_df.to_csv("./result/deepface_benchmark_gender_perturbed.csv", index=False)
result_benchmark_gender_perturbed = pd.read_csv('./result/deepface_benchmark_gender_perturbed.csv')

In [150]:
result_benchmark_gender_perturbed

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender_makeup_heavy,87.0,94.0,80.0,1000,Male,398,476,24,102,0
1,Facenet512,LFW_gender_makeup_heavy,86.0,99.0,73.0,1000,Female,366,498,2,134,0
2,Facenet,LFW_gender_makeup_heavy,67.0,100.0,35.0,1000,Male,174,500,0,326,0
3,Facenet,LFW_gender_makeup_heavy,69.0,100.0,38.0,1000,Female,192,500,0,308,0
4,VGG-Face,LFW_gender_makeup_heavy,73.0,70.0,81.0,1000,Male,404,327,173,96,0
5,VGG-Face,LFW_gender_makeup_heavy,83.0,96.0,69.0,1000,Female,346,487,13,154,0
6,OpenFace,LFW_gender_makeup_heavy,54.0,95.0,8.0,1000,Male,38,498,2,462,0
7,OpenFace,LFW_gender_makeup_heavy,56.0,100.0,12.0,1000,Female,60,500,0,440,0
8,DeepFace,LFW_gender_makeup_heavy,55.0,62.0,25.0,1000,Male,127,422,78,373,0
9,DeepFace,LFW_gender_makeup_heavy,58.0,79.0,22.0,1000,Female,110,471,29,390,0


### Etc testing